In [82]:
path = './ETH/'
path += "/seq_eth/obsmat.txt"
import os
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [19]:
csv_columns = ["frame_id", "id", "x", "z", "y", "vel_x", "vel_z", "vel_y"]
# read from csv => fill traj table
raw_dataset = pd.read_csv(path, sep=r"\s+", header=None, names=csv_columns)
raw_dataset["timestamp"]= raw_dataset.frame_id
start_frame = raw_dataset.frame_id.min()
d_frame = np.diff(pd.unique(raw_dataset["frame_id"]))
fps = d_frame[0] * 2.5  # 2.5 is the common annotation
for i in raw_dataset.index:
    raw_dataset.loc[i, "timestamp"] = (raw_dataset.loc[i, "frame_id"] - start_frame) / fps
raw_dataset = raw_dataset.loc[:, ["timestamp", "id", "x", "y"]]

unique agent 360


split data into train_id, testid

In [57]:
train_id, test_id = train_test_split(raw_dataset.id.unique(), test_size= 1 / 10, random_state=42)
train_id, test_id = set(train_id), set(test_id)

create dataset using, history_length, prediction_length

In [103]:
min_lenth = 10
history_length = 5
prediction_length = 2
total_sequence_length = history_length + prediction_length

train_dataset = []
test_dataset = []
for pid, group in raw_dataset.groupby("id"):
    timestamps = group["timestamp"].values
    x_values = group["x"].values
    y_values = group["y"].values

    for i in range(len(group) - total_sequence_length + 1):
        # Extract the sequence and target
        sequence = np.column_stack((x_values[i:i+history_length], y_values[i:i+history_length]))
        target = np.column_stack((x_values[i+history_length:i+total_sequence_length], y_values[i+history_length:i+total_sequence_length]))
        if pid in train_id:
            train_dataset.append((torch.tensor(sequence, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)))
        else:
            test_dataset.append((torch.tensor(sequence, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)))

# Convert to PyTorch DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# for batch_input, batch_target in train_loader:
#     # Check the shape of the input sequences in the first batch
#     print("Shape of input sequences:", batch_input.shape)

#     # Check the shape of the target sequences in the first batch
#     print("Shape of target sequences:", batch_target.shape)

#     # Assuming each sequence has two features (x, y)
#     num_features = batch_input.size(-1)
#     print("Number of features:", num_features)

#     # Assuming sequences have variable lengths
#     sequence_lengths = (batch_input != 0).sum(dim=1)
#     print("Sequence lengths:", sequence_lengths)

#     # Break after checking the first batch
#     break


In [117]:
# Assuming you have a simple LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])  # Assuming you want to predict from the last timestep
        return output
# Assuming 'input_size', 'hidden_size', and 'output_size' are appropriate values
input_size = 2  # Assuming 2 features (x, y) per timestep
hidden_size = 64
output_size = 2  # Assuming 2 features in the output
# Create an instance of the model
model = LSTMModel(input_size, hidden_size, output_size * prediction_length)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
# Assuming 'train_loader' is your DataLoader
for epoch in range(num_epochs):
    for batch_input, batch_target in train_loader:
        # Forward pass
        batch_input, batch_target = batch_input.to(device), batch_target.to(device)

        output = model(batch_input)
        output = output.view(-1, prediction_length, output_size)
        # Compute the loss
        loss = criterion(output, batch_target)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Print the loss after each epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
# Training complete

Epoch [1/10], Loss: 6.3642
Epoch [2/10], Loss: 1.6703
Epoch [3/10], Loss: 0.9372
Epoch [4/10], Loss: 0.2694
Epoch [5/10], Loss: 0.1391
Epoch [6/10], Loss: 0.1594
Epoch [7/10], Loss: 0.0637
Epoch [8/10], Loss: 0.0936
Epoch [9/10], Loss: 0.0654
Epoch [10/10], Loss: 0.0366


In [91]:
# Assuming 'test_loader' is your DataLoader
model.eval()  # Set the model to evaluation mode
total_loss = 0.0
num_samples = 0
# Disable gradient computation during evaluation
with torch.no_grad():
    for batch_input, batch_target in test_loader:
        batch_input, batch_target = batch_input.to(device), batch_target.to(device)
        # Forward pass
        output = model(batch_input)
        # Reshape the output to match the target shape
        output = output.view(-1, prediction_length, output_size)
        # Compute the loss
        loss = criterion(output, batch_target)
        total_loss += loss.item()
        num_samples += batch_input.size(0)
# Calculate the average loss
average_loss = total_loss / num_samples
print(f'Average Loss on Test Set: {average_loss:.4f}', num_samples)

Average Loss on Test Set: 0.0010 650


In [127]:
test_dataset[13]
test_a = [[7.4008, 5.3609],
         [6.9278, 5.4221],
         [6.6198, 5.3929],
         [6.1320, 5.3706],
         [5.5926, 5.3497]]

In [129]:
# test_a = test_dataset[13][0]
# Assuming 'test_a' is your single sequence with history length (x, y)
# Assuming 'model' is your trained LSTM model
# Preprocess the sequence (assuming it's a NumPy array)
test_a_tensor = torch.tensor(test_a, dtype=torch.float32).clone().detach().unsqueeze(0)  # Add batch dimension
# Ensure the model is in evaluation mode
model.eval()
# Make the prediction

with torch.no_grad():
    test_a_tensor = test_a_tensor.to(device)

    model_output = model(test_a_tensor)
# Reshape the output to match the target shape
model_output = model_output.view(-1, prediction_length, output_size)
# Extract the predicted values for the next steps

predicted_values = model_output[0, -prediction_length:, :].numpy()
predicted_values

array([[5.230974 , 5.1471486],
       [4.8149676, 5.107736 ]], dtype=float32)

In [119]:
torch.save(model.state_dict(), './model_weights.pth')

In [125]:
import torch

# Assuming 'ModelClass' is the class of your model (the same class used for training)
input_size = 2  # Assuming 2 features (x, y) per timestep
hidden_size = 64
output_size = 2  # Assuming 2 features in the output
# Create an instance of the model
model = LSTMModel(input_size, hidden_size, output_size * prediction_length)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the saved model weights
model.load_state_dict(torch.load('./model_weights.pth', map_location=device))

# Move the model to the GPU if available
model.to(device)

# Set the model to evaluation mode
test_a = test_dataset[13][0]
# Assuming 'test_a' is your single sequence with history length (x, y)
# Assuming 'model' is your trained LSTM model
# Preprocess the sequence (assuming it's a NumPy array)
test_a_tensor = torch.tensor(test_a, dtype=torch.float32).clone().detach().unsqueeze(0)  # Add batch dimension
# Ensure the model is in evaluation mode
model.eval()
# Make the prediction

with torch.no_grad():
    test_a_tensor = test_a_tensor.to(device)

    model_output = model(test_a_tensor)
# Reshape the output to match the target shape
model_output = model_output.view(-1, prediction_length, output_size)
# Extract the predicted values for the next steps

predicted_values = model_output[0, -prediction_length:, :].cpu().numpy()  # Move back to CPU for further processing if needed

predicted_values

/var/folders/fh/wkgzj0_d59xghrtzj8qr5prh0000gn/T/ipykernel_14116/4151446205.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_a_tensor = torch.tensor(test_a, dtype=torch.float32).clone().detach().unsqueeze(0)  # Add batch dimension


array([[5.2309985, 5.1471295],
       [4.8149967, 5.107718 ]], dtype=float32)